In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, Input, GlobalAveragePooling1D, Flatten, GRU, LayerNormalization
from keras.optimizers import Adam, SGD
from keras import Sequential
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from pyvi import ViTokenizer
from pyvi import ViUtils

# Data

### Load Data

In [ ]:
data = pd.read_csv("./data.csv")
data.head()

In [ ]:
sentiment_data = pd.DataFrame({'input': data['Comment Text'], 'label': data['Manual label']})
sentiment_data = sentiment_data.dropna()
sentiment_data = sentiment_data.reset_index(drop=True)
sentiment_data.head()

### Preprocessing Data

In [ ]:
input_data = sentiment_data['input'].values
input_label = sentiment_data['label'].values

label_dict = {'NEG':0, 'NEU':1, 'POS':2}
# label_dict = {0:"NEG", 1:"NEU", 2:"POS"}

input_pre = []
label_with_accent = []
for idx, dt in enumerate(input_data):
  input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(dt))
  input_text_pre = " ".join(input_text_pre)
  input_text_pre_no_accent = str(ViUtils.remove_accents(input_text_pre).decode("utf-8"))
  input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
  input_text_pre_no_accent = ViTokenizer.tokenize(input_text_pre_no_accent)
  input_pre.append(input_text_pre_accent)
  input_pre.append(input_text_pre_no_accent)
  label_with_accent.append(input_label[idx])
  label_with_accent.append(input_label[idx])

### PLot Dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for i, ax in enumerate(axes.ravel()):
    start_idx = i * 1000
    end_idx = (i + 1) * 1000
    seq_len = [len(j.split()) for j in input_pre[start_idx:end_idx]]
    pd.Series(seq_len).hist(bins=10, ax=ax)
    ax.set_title(f'From {start_idx} to {end_idx-1}')

plt.tight_layout()
plt.show()

In [ ]:
# seq_len = [len(i.split()) for i in input_pre[0:1000]]
# pd.Series(seq_len).hist(bins = 10, figsize=(20,15))
# plt.show()

# seq_len = [len(i.split()) for i in input_pre[1000:2000]]
# pd.Series(seq_len).hist(bins = 10)
# plt.show()

# seq_len = [len(i.split()) for i in input_pre[2000:3000]]
# pd.Series(seq_len).hist(bins = 10)
# plt.show()

# seq_len = [len(i.split()) for i in input_pre[3000:4000]]
# pd.Series(seq_len).hist(bins = 10)
# plt.show()

# seq_len = [len(i.split()) for i in input_pre[4000:5000]]
# pd.Series(seq_len).hist(bins = 10)

# plt.tight_layout()
# plt.show()

### Data shape

In [ ]:
label_idx = [i for i in label_with_accent]
label_tf = tf.keras.utils.to_categorical(label_idx, num_classes = 3, dtype = 'float32')

tokenizer_data = Tokenizer(oov_token = '<OOV>', filters = '', split = ' ')
tokenizer_data.fit_on_texts(input_pre)

tokenized_data_text = tokenizer_data.texts_to_sequences(input_pre)
vec_data = pad_sequences(tokenized_data_text, padding = 'post', maxlen = 800)

# Save model to pickle file
pickle.dump(tokenizer_data, open("tokenizer_data.pkl", "wb"))

print('input data.shape', vec_data.shape)
data_vocab_size = len(tokenizer_data.word_index)+1
print('data_vocab_size: ', data_vocab_size)


X_train, X_val, y_train, y_val = train_test_split(vec_data, label_tf, test_size = 0.2, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state = 42)
print('training sample: ', len(X_train))
print('validation sample: ', len(X_val))
print('test_sample: ', len(X_test))

# Model

### Architechture

In [ ]:
from keras.layers import MaxPooling1D,Conv1D, GlobalMaxPooling1D
from unicodedata import bidirectional
def generate_model():
    dropout_threshold = 0.4
    input_dim = data_vocab_size
    output_dim = 50
    input_length = 800
    initializer = tf.keras.initializers.GlorotNormal()

    input_layer = Input(shape = (input_length))
    feature = Embedding(input_dim = input_dim, output_dim = output_dim, input_length = input_length, embeddings_initializer = 'GlorotNormal')(input_layer)

    cnn_feature = Conv1D(filters = 50, kernel_size = 3, padding = 'same', activation = 'relu')(feature)
    cnn_feature = MaxPooling1D()(cnn_feature)
    cnn_feature = Dropout(dropout_threshold)(cnn_feature)
    cnn_feature = Conv1D(filters = 50, kernel_size = 3, padding = 'same', activation = 'relu')(cnn_feature)
    cnn_feature = MaxPooling1D()(cnn_feature)
    cnn_feature = LayerNormalization()(cnn_feature)
    cnn_feature = Dropout(dropout_threshold)(cnn_feature)


    bi_lstm_feature = Bidirectional(LSTM(units = 50, dropout = dropout_threshold, return_sequences = True, kernel_initializer = initializer), merge_mode = 'concat')(feature)
    bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)

    bi_lstm_feature = Bidirectional(GRU(units = 50, dropout = dropout_threshold, return_sequences = True, kernel_initializer = initializer), merge_mode = 'concat')(bi_lstm_feature)
    bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)
    bi_lstm_feature = LayerNormalization()(bi_lstm_feature)

    combine_feature = tf.keras.layers.Concatenate()([cnn_feature, bi_lstm_feature])
    combine_feature = GlobalMaxPooling1D()(combine_feature)
    combine_feature = LayerNormalization()(combine_feature)

    classifier = Dense(90, activation = 'relu')(combine_feature)
    classifier = Dropout(0, 2)(classifier)
    classifier = Dense(70, activation = 'relu')(classifier)
    classifier = Dropout(0, 2)(classifier)
    classifier = Dense(50, activation = 'relu')(classifier)
    classifier = Dropout(0, 2)(classifier)
    classifier = Dense(30, activation = 'relu')(classifier)
    classifier = Dropout(0, 2)(classifier)
    classifier = Dense(3, activation = 'softmax')(classifier)

    model = tf.keras.Model(inputs = input_layer, outputs = classifier)

    return model

model = generate_model()
adam = Adam(learning_rate = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

### Architechture Visualization

In [ ]:
dot_img_file = "./images/model_visualize.png"
tf.keras.utils.plot_model(model, to_file = dot_img_file, show_shapes = True)

### Train

In [ ]:
callback_model = tf.keras.callbacks.ModelCheckpoint('model_cnn_bilstm.h5', monitor = 'val_loss')
history = model.fit(x = X_train, y = y_train, validation_data = (X_val, y_val), epochs = 10, callbacks = [callback_model])

### Validation

In [ ]:
model.load_weights('model_cnn_bilstm.h5')
model.evaluate(X_test, y_test)

# Running

In [ ]:
def preprocess_raw_input(raw_input, tokenizer):
    input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
    input_text_pre = ' '.join(input_text_pre)
    input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
    print('Text preprocessed: ',input_text_pre_accent)
    tokenized_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
    vec_data = pad_sequences(tokenized_data_text, padding = 'post', maxlen = 800)
    return vec_data

def inference_model(input_feature, model):
    output = model(input_feature).numpy()[0]
    label_list = ['NEG', 'NEU', 'POS']
    res = dict(zip(label_list, output))
    result = output.argmax()
    conf = float(output.max())
    label_dict = {'NEG': 0, 'NEU': 1, 'POS': 2}
    label = list(label_dict.keys())
    return label[int(result)], conf, res


def prediction(raw_input, tokenizer, model):
    input_model = preprocess_raw_input(raw_input, tokenizer_data)
    result, conf,res = inference_model(input_model, model)

    return result, conf, res

my_model = generate_model()
my_model = load_model('model_cnn_bilstm.h5')

with open(r'tokenizer_data.pkl', 'rb') as input_file:
    my_tokenizer = pickle.load(input_file)

while(True):
    text = input()
    if text == 'end':
        break
    else:
        print(str(prediction(text, my_tokenizer, my_model)) + '\n')